# Report generation

Apply group chat conversation arquitecture between several agents to generate a report about the future of artificial intelligence in everyday life.

### Setup

In [ ]:
from dotenv import load_dotenv
import os

# Load .env file
load_dotenv(override=True)

# Get environment variables
openai_key = os.getenv("OPENAI_KEY")
openai_endpoint = os.getenv("OPENAI_ENDPOINT")

In [ ]:
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient

az_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment="gpt-4o-mini",
    model="gpt-4o-mini",
    api_version="2024-06-01",
    azure_endpoint=openai_endpoint,
    api_key=openai_key
)

### Define the task

In [ ]:
task = "Make a report about the future of artificial intelligence in everyday life"
# Use this to show how the team of agents is organized to only use the ethic agent, for example -> autonomy
#task = "Make a report about the ethic use of artificial intelligence"

### Define group of agents

In [48]:
from autogen_agentchat.agents import AssistantAgent

planning_agent = AssistantAgent(
    "PlanningAgent",
    description="An agent for planning tasks, this agent should be the first to engage when given a new task.",
    model_client=az_model_client,
    system_message="""
    You are a planning agent.
    Your job is to break down complex tasks into smaller, manageable subtasks.
    Your team members are:
        AIApplicationsSpecialist: Analyzes real-world AI applications in daily life
        AIEthicsAdvisor: Evaluates AI ethics, privacy, and bias issues
        AIBusinessJobsAnalyst: Examines AI impact on industries and jobs
        Writer: A writer tasked with compiling the conversation highlights and generating the final report
        
    You only plan and delegate tasks - you do not execute them yourself.
    
    Use only the necessary team members to solve the task, you do not need to use all of them always.

    When assigning tasks, use this format:
    1. <agent> : <task>
    """,
)

In [49]:
ai_applications_agent = AssistantAgent(
    name="AIApplicationsSpecialist",
    description="Analyzes real-world AI applications in daily life.",
    system_message="""You specialize in AI-powered tools such as smart assistants, smart homes, 
    healthcare AI, and workplace automation. Explain how AI enhances convenience, efficiency, 
    and decision-making in these areas.
    Focus only on your assigned aspect and do not attempt to answer questions or solve tasks 
    outside your scope.""",
    model_client=az_model_client,
)

ai_ethics_agent = AssistantAgent(
    name="AIEthicsAdvisor",
    description="Evaluates AI ethics, privacy, and bias issues.",
    system_message="""You analyze AI risks like bias, transparency, misinformation, and data security. 
    Propose responsible AI practices to ensure fairness and trustworthiness.
    Focus only on your assigned aspect and do not attempt to answer questions or solve tasks 
    outside your scope.""",
    model_client=az_model_client,
)

ai_industry_agent = AssistantAgent(
    name="AIBusinessJobsAnalyst",
    description="Examines AI’s impact on industries and jobs.",
    system_message="""You assess how AI transforms industries, automates tasks, and reshapes the job market. 
    Identify key disruptions and how businesses adapt.
    Focus only on your assigned aspect and do not attempt to answer questions or solve tasks 
    outside your scope.""",
    model_client=az_model_client,
)

In [50]:
writer_agent = AssistantAgent(
    name="Writer",
    description="A writer tasked with compiling the conversation highlights and generating the final report.",
    system_message="""You are a writer tasked with summarizing the highlights from a conversation 
    between various experts. After the conversation ends, your job is to compile the key takeaways 
    from each participant, provide an overview of the discussion, and generate a well-organized report. 
    Add the word TERMINATE at the end of the report.""",
    model_client=az_model_client,
)

### Define Group Chat
Use SelectorGroupChat for LLM-based speaker selection.

In [51]:
selector_prompt = """Select an agent to perform task.

{roles}

Current conversation context:
{history}

Read the above conversation, then select an agent from {participants} to perform the next task.
Make sure the planner agent has assigned tasks before other agents start working.
Only select one agent.
"""

In [52]:
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.conditions import TextMentionTermination, MaxMessageTermination
from autogen_agentchat.ui import Console

termination = TextMentionTermination("TERMINATE") | MaxMessageTermination(8)
team = SelectorGroupChat(
    [ai_applications_agent, planning_agent, ai_ethics_agent, ai_industry_agent, writer_agent],
    model_client = az_model_client,
    termination_condition = termination,
    selector_prompt = selector_prompt
)

# `run_stream` returns an async generator to stream the intermediate messages.
stream = team.run_stream(task=task)
# `Console` is a simple UI to display the stream.
await Console(stream)

---------- user ----------
Make a report about the ethic use of artificial intelligence
---------- PlanningAgent ----------
1. AIEthicsAdvisor: Research and analyze current ethical issues surrounding artificial intelligence, including privacy concerns, bias in AI algorithms, and accountability in AI decision-making.
2. Writer: Compile the findings from the AIEthicsAdvisor and generate a report outlining the ethical use of artificial intelligence, including recommendations for best practices.
---------- AIEthicsAdvisor ----------
I will focus on the aspect of bias in AI algorithms and propose responsible AI practices to ensure fairness and trustworthiness.

### Addressing Bias in AI Algorithms: Responsible Practices

#### Understanding Bias in AI
Bias in AI can manifest in various forms, including data bias, algorithmic bias, and human bias. These biases can lead to unfair outcomes and reinforce existing societal inequalities. Addressing bias is crucial to achieving equity and preventin

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Make a report about the ethic use of artificial intelligence', type='TextMessage'), TextMessage(source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=181, completion_tokens=66), content='1. AIEthicsAdvisor: Research and analyze current ethical issues surrounding artificial intelligence, including privacy concerns, bias in AI algorithms, and accountability in AI decision-making.\n2. Writer: Compile the findings from the AIEthicsAdvisor and generate a report outlining the ethical use of artificial intelligence, including recommendations for best practices.', type='TextMessage'), TextMessage(source='AIEthicsAdvisor', models_usage=RequestUsage(prompt_tokens=147, completion_tokens=608), content="I will focus on the aspect of bias in AI algorithms and propose responsible AI practices to ensure fairness and trustworthiness.\n\n### Addressing Bias in AI Algorithms: Responsible Practices\n\n#### Understanding 